A feature_engine é uma biblioteca de Python voltada para engenharia de atributos em projetos de machine learning. Ela ajuda a preparar variáveis de forma organizada, sem precisar escrever tudo manualmente.

Usar, por exemplo, para tratamento de valores faltantes: média, mediana, constantes, métodos por categoria

In [0]:
!pip install feature_engine

MÓDULO DE IMPUTAÇÃO E SELEÇÃO!

In [0]:
from feature_engine import imputation, selection

#esc + A acidiona um novo bloco de código

In [0]:
# SEMMA

# SAMPLE

df = spark.table("analitics.olist.abt_ativacao").toPandas()
df.display()

In [0]:
df.head(5)

In [0]:
df['dtRef'].unique()

In [0]:
df_oot = df[df['dtRef'] == df['dtRef'].max()].copy()

df_train = df[df['dtRef'] < df['dtRef'].max()].copy()

In [0]:
from sklearn import model_selection
features = df_train.columns[4:].tolist()
target = 'flagChurn'

X = df_train[features] #se matriz usa maiusculo
y = df_train[target] # vetor usa minusculo

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, 
                                                                    random_state=42, #semente aleatoria
                                                                    test_size=0.2, #pegar 20% pra teste
                                                                    stratify=y) #olha a variavel resposta e vai sorteando de forma proporcional

print("Taxa Treino:", y_train.mean())
print("Taxa Teste:", y_test.mean())

In [0]:
X_train

In [0]:
# Identificação de Missings

train_isna = X_train.isna().sum()
train_isna[train_isna>0]

IMPUTAR POR **MÉDIA**!

In [0]:
# EXPLORE/ MODIFY
# MISSING


train_isna = X_train.isna().mean()
train_isna[train_isna>0]

In [0]:
help(imputation.MeanMedianImputer)


In [0]:
# Declaro o que será a imputação (mas não faz ainda a imputação)

# Média
# Lista de variáveis
features_imput_mean = ['mediaCubagemProduto',
    'mediaAvaliacao168d',
    'mediaAvaliacao336d',
    'mediaAvaliacao',
    'mediaTamanhoNomeProduto',
    'mediaTamanhoDescricaoProduto', 
    'mediaPesoGramas',]

imputer_mean = imputation.MeanMedianImputer(imputation_method='mean', variables=features_imput_mean)

In [0]:
X_train[features_imput_mean].isna().sum()

In [0]:
# Ensinando o que é média
# fit é a palavrinha mágica para aprender
imputer_mean.fit(X_train, y_train)

#Agora vamos imputar!

X_train_transform = imputer_mean.transform(X_train)

X_test_transform = imputer_mean.transform(X_test) # com a média de treino que é a única que ele conhece

X_train_transform[features_imput_mean].isna().sum()

IMPUTAR POR **ZERO**!

PASSOS: 
1) LISTA DE VARIÁVES
2) CRIAR O OBJETO
3) FIT (ENSINAR O QUE É ZERO)
4) IMPUTAR O ZERO COM O TRANSFORM

In [0]:
# Zeros
features_imput_0 = [
    'categoriasDistintas',
    'qtdMediaFotos',
    'pctProdutosComFotos',
    'pctProdutosComDescricao',
    'cliUfAC',
    'cliUfAL',
    'cliUfAM',
    'cliUfAP',
    'cliUfBA',
    'cliUfCE',
    'cliUfDF',
    'cliUfES',
    'cliUfGO',
    'cliUfMA',
    'cliUfMG',
    'cliUfMS',
    'cliUfMT',
    'cliUfPA',
    'cliUfPB',
    'cliUfPE',
    'cliUfPI',
    'cliUfPR',
    'cliUfRJ',
    'cliUfRN',
    'cliUfRO',
    'cliUfRR',
    'cliUfRS',
    'cliUfSC',
    'cliUfSE',
    'cliUfSP',
    'cliUfTO',
    'pctNota1',
    'pctNota2',
    'pctNota3',
    'pctNota4',
    'pctNota5',
    'pctNotaBaixa',
    'maiorCategoriaTotalPedidos',
    'shareTopCategorias',
    'pctautomotivo',
    'pctbeleza_saude',
    'pctbrinquedos',
    'pctcama_mesa_banho',
    'pctcool_stuff',
    'pcteletroportateis',
    'pctesporte_lazer',
    'pctferramentas_jardim',
    'pctinformatica_acessorios',
    'pctmoveis_decoracao',
    'pctmoveis_escritorio',
    'pctperfumaria',
    'pctrelogios_presentes',
    'pcttelefonia',
    'pctutilidades_domesticas',
    'pctPedidosEntreguesNoPrazo',
    'qtdCliente',
    'cliNovo',
    'cliPontual',
    'cliRecorrente',
    'qtdProdutosDistintos',
    'maiorCategoriaTotalReceita',
]

imputer_zeros = imputation.ArbitraryNumberImputer(arbitrary_number=0, variables=features_imput_0)

In [0]:
# Ensinando o que é zero
imputer_zeros.fit(X_train, y_train)


# Aqui transforma
X_train_transform = imputer_zeros.transform(X_train_transform)
X_test_transform = imputer_zeros.transform(X_test_transform) # com o transform que é pra ir empilhando

X_train_transform[features_imput_0].isna().sum()

IMPUTAR COM **1**!

LEMBRAR QUE ESTAMOS EMPILHANDO AS TRANFORMAÇÕES, LOGO A TRASFORMAÇÃO VAI DE ACORDO COM A TRANSFORMAÇÃO ANTERIOR


In [0]:
features_imput_1 = [
'pctTendencia1m_2m',
'pctTendencia1m_3m',
'pctTendencia1m_4m',
'pctTendencia1m_5m',
'pctTendencia1m_6m',
'pctTendencia1m_12m',
'mediaQtdeParcelas',
]

imputer_ones = imputation.ArbitraryNumberImputer(arbitrary_number=1, variables=features_imput_1)

In [0]:
# Ensinando o que é UM
imputer_ones.fit(X_train, y_train)


# Aqui transforma
X_train_transform = imputer_ones.transform(X_train_transform)
X_test_transform = imputer_ones.transform(X_test_transform) # com o transform que é pra ir empilhando

X_train_transform[features_imput_1].isna().sum()

IMPUTAR O QUE É **MISSING**!

In [0]:
flag_missing = [
'maiorCategoriaVenda',
'maiorCategoriaQtdePedidos',
'vendedorRegiao',
]

imputer_flag_missing = imputation.CategoricalImputer(fill_value="MISSING", variables=flag_missing)

imputer_flag_missing.fit(X_train, y_train)

X_train_transform = imputer_flag_missing.transform(X_train_transform)
X_test_transform = imputer_flag_missing.transform(X_test_transform)

X_train_transform.isna().sum().sort_values(ascending=False).head(15)

IMPUTAR O QUE É **MAXIMO**!

Esse trecho cria um transformador do Feature Engine para imputar outliers ou valores faltantes usando o limite máximo permitido para a cauda direita da distribuição.

Aqui está o que cada parâmetro significa:

EndTailImputer
É um imputador que substitui valores ausentes ou outliers por um ponto de corte calculado na cauda da distribuição.

imputation_method = 'max'
O imputador calcula o limite superior da variável e usa esse valor para imputar.
Esse limite é definido como:
máximo permitido = valor do limite da cauda direita.

tail = 'right'
Indica que o imputador vai olhar para a cauda direita da distribuição, ou seja, valores muito altos.

fold = 1
Controla o quão distante o limite será da estatística base.
No caso da cauda direita, normalmente usa:
limite = Q3 + fold * IQR
fold controla o tamanho desse limite.

variables = features_imput_max
Lista de variáveis numéricas onde o imputador será aplicado.

O que isso significa na prática

Para cada variável em features_imput_max:

O EndTailImputer calcula o limite superior permitido usando Q3 e IQR.

Valores acima desse limite ou valores ausentes são substituídos pelo próprio limite.

É útil quando você quer reduzir o impacto de valores extremos, mantendo coerência estatística sem simplesmente usar mediana ou média.

In [0]:
features_imput_max = ['qtdeMediaDiasEntrePedidos']

imputer_max = imputation.EndTailImputer(imputation_method='max', tail='right', fold=1, variables=features_imput_max)

imputer_max.fit(X_train, y_train)
X_train_transform = imputer_max.transform(X_train_transform)
X_test_transform = imputer_max.transform(X_test_transform)

X_train_transform.isna().sum().sort_values(ascending=False).head(15)

SELECIONAR AS VARIÁVEIS QUE SERÃO **DROPADOS**!

In [0]:
features_drop = ['crescimentoD28',
                 'mediaAvaliacao28d',
                 'mediaAvaliacao56d',
                 'mediaAvaliacao84d']

drop_step = selection.DropFeatures(features_to_drop = features_drop)

drop_step.fit(X_train, y_train)
X_train_transform = drop_step.transform(X_train_transform)
X_test_transform = drop_step.transform(X_test_transform)

X_train_transform.isna().sum().sort_values(ascending=False).head(15)


In [0]:
import pandas as pd

pd.set_option('display.max_rows', 500)

# x
df_analysis = X_train_transform.copy()

# Y
df_analysis[target] = y_train

# Seleciona apenas colunas numéricas (exclui tipos object e strings).
df_analysis.select_dtypes(exclude='object').groupby(target).median().T

In [0]:
df_analysis.select_dtypes(include='object').columns

Calculando a média da variável target dentro de cada categoria da variável na posição 0

In [0]:
cat_vars_analysis = df_analysis.select_dtypes(include='object').columns.tolist()

df_analysis.groupby(cat_vars_analysis[0])[target].mean()

Esses números significam:

19% dos itens da categoria "Crescente" têm target = 1

18% dos itens da categoria "Decrescente" têm target = 1

55% dos itens da categoria "Estável" têm target = 1

Eles não representam uma divisão do total, por isso não somam 100%.

In [0]:
df_analysis.groupby(cat_vars_analysis[1])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[2])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[3])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[4])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[5])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[6])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[7])[target].mean()

In [0]:
df_analysis.groupby(cat_vars_analysis[8])[target].mean().sort_values()

In [0]:
df_analysis.groupby(cat_vars_analysis[9])[target].mean().sort_values()

Fazendo de uma vez para todas as variáveis categóricas.

In [0]:
for n in cat_vars_analysis:
        print(f"Feature: {n}")
        print(df_analysis.groupby(n)[target].mean())
        print("\n","-"*20, end="\n\n")
